In [4]:
using Flux, Plots, Statistics, MLDatasets, Images, TimeSeries

In [7]:
using CSV, DataFrames
df = CSV.read("BAJAJFINSV.csv", DataFrame)

Row,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
,Date,String15,String3,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64?,Int64,Float64
1,2008-05-26,BAJAJFINSV,EQ,2101.05,600.0,619.0,501.0,505.1,509.1,548.85,3145446,1.72637e14,missing,908264,0.2888
2,2008-05-27,BAJAJFINSV,EQ,509.1,505.0,610.95,491.1,564.0,554.65,572.15,4349144,2.48837e14,missing,677627,0.1558
3,2008-05-28,BAJAJFINSV,EQ,554.65,564.0,665.6,564.0,643.0,640.95,618.37,4588759,2.83753e14,missing,774895,0.1689
4,2008-05-29,BAJAJFINSV,EQ,640.95,656.65,703.0,608.0,634.5,632.4,659.6,4522302,2.98292e14,missing,1006161,0.2225
5,2008-05-30,BAJAJFINSV,EQ,632.4,642.4,668.0,588.3,647.0,644.0,636.41,3057669,1.94593e14,missing,462832,0.1514
6,2008-06-02,BAJAJFINSV,EQ,644.0,650.0,699.0,622.0,687.0,686.95,670.51,2247399,1.50691e14,missing,351112,0.1562
7,2008-06-03,BAJAJFINSV,EQ,686.95,672.0,689.8,632.55,670.55,672.05,665.87,2299383,1.53108e14,missing,391772,0.1704
8,2008-06-04,BAJAJFINSV,EQ,672.05,674.0,674.0,566.6,595.0,598.95,602.36,2832725,1.70632e14,missing,735765,0.2597
9,2008-06-05,BAJAJFINSV,EQ,598.95,603.0,639.8,572.3,631.5,631.85,610.65,1344712,8.21152e13,missing,167819,0.1248


In [18]:
vwap = df.VWAP

3201-element Vector{Float64}:
   548.85
   572.15
   618.37
   659.6
   636.41
   670.51
   665.87
   602.36
   610.65
   628.44
   624.95
   653.65
   662.71
     ⋮
  9587.77
  9694.01
  9823.88
  9501.23
  9824.26
  9746.74
  9956.69
  9995.72
 10036.76
 10445.96
 10980.4
 11081.78

In [16]:

m = Flux.Chain(
            LSTM(df.VWAP, 10),
            Dense(10, 1))
            Chain(Recur(LSTMCell(3210, 10))), 
            Dense(10, 1))

LoadError: MethodError: no method matching Flux.LSTMCell(::Vector{Float64}, ::Int64)
[0mClosest candidates are:
[0m  Flux.LSTMCell(::I, ::H, [91m::V[39m, [91m::S[39m) where {I, H, V, S} at /Users/irene/.julia/packages/Flux/uCLgc/src/layers/recurrent.jl:295
[0m  Flux.LSTMCell([91m::Integer[39m, ::Integer; kw...) at /Users/irene/.julia/packages/Flux/uCLgc/src/deprecations.jl:70

In [23]:
# https://fluxml.ai/Flux.jl/stable/models/recurrence/
m = Flux.Chain(RNN(2 => 5), Dense(5 => 1))
Chain(
  Flux.Recur(
    Flux.RNNCell(2 => 5, tanh),              # 45 parameters
  ),
  Dense(5 => 1),                        # 6 parameters
)

Chain(
  Recur(
    RNNCell(2 => 5, tanh),              # 45 parameters
  ),
  Dense(5 => 1),                        # 6 parameters
)         # Total: 6 trainable arrays, 51 parameters,
          # plus 1 non-trainable, 5 parameters, summarysize 580 bytes.